<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<br><b>Little Demo: Lightcurve of a Type Ia Supernova</b> <br>
Contact author: Melissa Graham <br>
Last verified to run: 2023-09-23 <br>
LSST Science Pipelines version: Weekly 2023_37 <br>

Plot the multi-band lightcurve of one Type Ia supernova.

Import packages.

In [ ]:
import numpy
import matplotlib.pyplot as plt
from lsst.rsp import get_tap_service

Start the TAP service.

In [ ]:
service = get_tap_service("tap")

Set plot style parameters.

In [ ]:
plt.style.use('tableau-colorblind10')
filter_names = ['u', 'g', 'r', 'i', 'z', 'y']
filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                 'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

This demo assumes that the RA, Dec of the supernova is already known.

In [ ]:
sn_coords = '68.880903, -43.571043'

Find the `DiaObject` identifier for the supernova, using its coordinates.

This query searches for objects detected in difference images within 1 arcsecond
of the supernova's coordinates.

In [ ]:
DiaObjs = service.search("SELECT ra, decl, diaObjectId, nDiaSources "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE CONTAINS(POINT('ICRS', ra, decl), "
                         "CIRCLE('ICRS', " + sn_coords + ", 0.0003)) = 1 ").to_table()

View table.

In [ ]:
DiaObjs

Save the `diaObjectId` for the supernova of interest.

In [ ]:
sn_diaObjectId = 1252792344780999776

Retrieve data from the `ForcedSourceOnDiaObject` table for this `DiaObject`. 
Return filter (`band`) and difference-image forced PSF-fit flux (`psfDiffFlux`) 
and its error (`psfDiffFluxErr`).
Join with the `ccdVisitId` table to get the time of the observation.

**This query will return forced photometry for *all* LSST visits.**
(Query the `DiaSource` table instead of `ForcedSourceOnDiaObject` if only
SNR $>5$ detections are desired.)

In [ ]:
DiaSrcs = service.search("SELECT fsodo.band, fsodo.psfDiffFlux, fsodo.psfDiffFluxErr, "
                         "cv.expMidptMJD "
                         "FROM dp02_dc2_catalogs.ForcedSourceOnDiaObject AS fsodo "
                         "JOIN dp02_dc2_catalogs.CcdVisit AS cv "
                         "ON cv.ccdVisitId = fsodo.ccdVisitId "
                         "WHERE fsodo.diaObjectId = "+str(sn_diaObjectId)).to_table()
print(len(DiaSrcs))

Option to view table.

In [ ]:
# DiaSrcs

Plot the forced-photometry lightcurve in flux, not magnitude.
Fluxes are from difference images and can be negative. 

In [ ]:
fig = plt.figure(figsize=(6, 4))
for filt in filter_names:
    fx = numpy.where(DiaSrcs['band'] == filt)[0]
    plt.plot(DiaSrcs['expMidptMJD'][fx], DiaSrcs['psfDiffFlux'][fx],
             filter_symbols[filt], ms=15, mew=0, alpha=0.5, 
             color=filter_colors[filt])
plt.xlabel('Modified Julian Date')
plt.ylabel('Difference-Image Flux [nJy]')
plt.title('Forced-Photometry SNIa Lightcurve')
plt.show()

Plot the forced-photometry lightcurve in magnitudes, only for epochs with SNR>5 detections.

In [ ]:
fig = plt.figure(figsize=(6, 4))
for filt in filter_names:
    fx = numpy.where(DiaSrcs['band'] == filt)[0]
    tx = numpy.where(DiaSrcs['psfDiffFlux'][fx] > 5.0 * DiaSrcs['psfDiffFluxErr'][fx])[0]
    plt.plot(DiaSrcs['expMidptMJD'][fx[tx]],
             -2.5 * numpy.log10(DiaSrcs['psfDiffFlux'][fx[tx]]) + 31.4,
             filter_symbols[filt], ms=15, mew=0, alpha=0.5, 
             color=filter_colors[filt])
plt.xlabel('Modified Julian Date')
plt.ylabel('Difference-Image Magnitude')
plt.title('SNR>5 SNIa Lightcurve')
plt.ylim([25, 22])
plt.show()

Recall that the DC2 simulation used an old baseline observing strategy without rolling cadence.